![](https://img.shields.io/pypi/l/skmixed)
![](https://img.shields.io/github/workflow/status/aksholokhov/skmixed/Testing%20and%20Coverage/sr3)
![](https://img.shields.io/readthedocs/skmixed)
![](https://img.shields.io/codecov/c/github/aksholokhov/skmixed/sr3?flag=unittests)
[![Codacy Badge](https://app.codacy.com/project/badge/Grade/749695b3c6fd43bb9fdb499ec0ace67b)](https://www.codacy.com/gh/aksholokhov/skmixed/dashboard?utm_source=github.com&amp;utm_medium=referral&amp;utm_content=aksholokhov/skmixed&amp;utm_campaign=Badge_Grade)

# SR3: Features Selection with Relaxed Regularization Framework.

# Quickstart

SR3 is a library designed for feature selection via Sparse Relaxed Regularized Regression (SR3) framework.

## Installation

In [ ]:
# !pip install skmixed

## Usage

### Linear Models

In [ ]:
import numpy as np

from skmixed.linear.problems import LinearProblem

# Create a sample dataset
seed = 42
num_objects = 300
num_features = 500
np.random.seed(seed)
# create a vector of true model's coefficients
true_x = np.random.choice(2, size=num_features, p=np.array([0.9, 0.1]))
# create sample data
a = 10*np.random.randn(num_objects, num_features)
b = a.dot(true_x) + np.random.randn(num_objects)

print(f"The dataset has {a.shape[0]} objects and {a.shape[1]} features; \n"
      f"The vector of true parameters contains {sum(true_x != 0)} non-zero elements out of {num_features}.")

In [ ]:
# Automatic features selection using information criterion
from skmixed.linear.models import LinearL1ModelSR3
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.fixes import loguniform

model = LinearL1ModelSR3(logger_keys=('aic', 'bic'))
params = {
    #"el": loguniform(1e-2, 1e2),
    "lam": loguniform(1e-1, 1e2)
}
selector = RandomizedSearchCV(estimator=model,
                              param_distributions=params,
                              n_iter=50,
                              scoring=lambda clf, X, y: -clf.logger_.get('bic'))

selector.fit(a, b)
maybe_x = selector.best_estimator_.coef_['x']
tn, fp, fn, tp = confusion_matrix(true_x, maybe_x != 0).ravel()

print(f"The model found {tp} out of {tp + fn} features correctly, but also chose {fp} extra irrelevant features. \n"
      f"The best parameter is {selector.best_params_}")

Linear Mixed-Effects Models